# Getting DLR Data

In [ ]:
import rioxarray as rxr
import xarray as xr
from odc.stac import load
import pystac_client
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import numpy as np
from shapely.geometry import Point
from concurrent.futures import ThreadPoolExecutor, as_completed
from tqdm import tqdm
import os

In [ ]:
# Create cache directory
CACHE_DIR = "stac_data_cache"
os.makedirs(CACHE_DIR, exist_ok=True)

In [ ]:
def create_bbox(lon, lat, step=0.000001):
    """Create a bounding box around a point."""
    return [lon - step, lat - step, lon + step, lat + step]

def get_stac_data_for_point(args):
    """Process a single point (for parallel execution)"""
    catalog, collection, measurements, point_idx, lon, lat, bbox_step = args
    
    # Check if cached result exists
    cache_file = f"{CACHE_DIR}/point_{lon}_{lat}.parquet"
    if os.path.exists(cache_file):
        try:
            return pd.read_parquet(cache_file)
        except Exception as e:
            print(f"Error reading cache file {cache_file}: {e}")
            pass  # If cache read fails, continue with regular processing
    
    try:
        # Create bounding box for this point
        bbox = create_bbox(lon, lat, bbox_step)
        
        # Search for items
        search = catalog.search(
            collections=collection,
            bbox=bbox,
            datetime="2018-03-01/2020-12-31"
        )
        
        # Convert search results to list
        items = list(search.items())
        
        if len(items) > 0:
            # Load the data
            dataset = load(
                items,
                measurements=measurements,
                bbox=bbox,
                resolution=20
            )
            
            # Convert to dataframe
            data_point = dataset.isel(time=0).to_dataframe().reset_index()
            
            # Add point metadata
            data_point['point_index'] = point_idx
            data_point['source_lon'] = lon
            data_point['source_lat'] = lat
            
            # Cache the result
            try:
                data_point.to_parquet(cache_file)
            except Exception as e:
                print(f"Error caching point {point_idx}: {e}")
                pass  # If caching fails, continue anyway
            
            return data_point
        else:
            print(f"No items found for point {point_idx} ({lon}, {lat})")
            return None
    except Exception as e:
        print(f"Error processing point {point_idx}: {e}")
        return None

def get_all_auxiliary_data(catalog, collection, measurements, long_lat, bbox_step=0.000001, max_workers=4):
    """Retrieve STAC data for all points using parallel processing."""
    
    # Prepare arguments for each point
    args_list = []

    # Create argument list for all points
    for i in range(len(long_lat)): 
        args_list.append((
            catalog, 
            collection, 
            measurements, 
            i,  # Point index
            long_lat.iloc[i]['GPS_LONG'],
            long_lat.iloc[i]['GPS_LAT'],
            bbox_step
        ))
    
    # Process points in parallel
    results = []
    with ThreadPoolExecutor(max_workers=max_workers) as executor:
        # Submit all tasks and track with progress bar
        futures = [executor.submit(get_stac_data_for_point, args) for args in args_list]
        
        for future in tqdm(as_completed(futures), total=len(args_list), desc="Processing points"):
            result = future.result()
            if result is not None:
                results.append(result)
    
    # Combine all results
    if not results:
        print("No data retrieved!")
        return None
    
    points_df = pd.concat(results, ignore_index=True)
    
    # Create geometry points for GeoDataFrame
    geometry_points = [Point(x, y) for x, y in zip(points_df['x'], points_df['y'])]
    
    # Convert to GeoDataFrame
    points_gdf = gpd.GeoDataFrame(points_df, geometry=geometry_points, crs=3035)
    
    return points_gdf

In [ ]:
# dlr_measurements = ["MREF_B02", "MREF_B03", "MREF_B04", "MREF_B05", "MREF_B06", "MREF_B07", "MREF_B08", "MREF_B8A", "MREF_B11", "MREF_B12",  # Mean Reflectance bands
#                         "MREF-STD_B02", "MREF-STD_B03", "MREF-STD_B04", "MREF-STD_B05", "MREF-STD_B06", "MREF-STD_B07", "MREF-STD_B08", "MREF-STD_B8A", "MREF-STD_B11", "MREF-STD_B12",  # Standard deviation bands
#                         "SRC_B02", "SRC_B03", "SRC_B04", "SRC_B05", "SRC_B06", "SRC_B07", "SRC_B08", "SRC_B8A", "SRC_B11", "SRC_B12",  # Bare Surface Reflectance bands
#                         "SRC-STD_B02", "SRC-STD_B03", "SRC-STD_B04", "SRC-STD_B05", "SRC-STD_B06", "SRC-STD_B07", "SRC-STD_B08", "SRC-STD_B8A", "SRC-STD_B11", "SRC-STD_B12",  # Bare Surface Standard Deviation bands
#                         "SRC-CI95_B02", "SRC-CI95_B03", "SRC-CI95_B04", "SRC-CI95_B05", "SRC-CI95_B06", "SRC-CI95_B07", "SRC-CI95_B08", "SRC-CI95_B8A", "SRC-CI95_B11", "SRC-CI95_B12",  # Bare Surface 95% Confidence Interval bands
#                         "SFREQ-BSF" #, "SFREQ-BSC", "SFREQ-VPC"
#                         ]

In [ ]:
# Load your data
target_raw = pd.read_csv('data/France_lab.csv')
long_lat = target_raw[['GPS_LONG', 'GPS_LAT']]

# Initialize STAC catalog
dlr_catalog = pystac_client.Client.open("https://geoservice.dlr.de/eoc/ogc/stac/v1")

# Define measurements (you can reduce this list if you don't need all bands)
dlr_measurements = ["MREF_B02", "MREF_B03", "MREF_B04", "MREF_B08", "MREF_B11", "MREF_B12"]

# Define collection
dlr_collection = ["S2-soilsuite-europe-2018-2022-P5Y"]

In [ ]:
# Process all points (consider using a subset for testing: long_lat.iloc[:10])
results = get_all_auxiliary_data(
    catalog=dlr_catalog,
    collection=dlr_collection,
    measurements=dlr_measurements,
    long_lat=long_lat,
    bbox_step=0.000001,
    max_workers=4  # Adjust based on your CPU and bandwidth
)

# Save the results
if results is not None:
    results.to_file("data/auxiliary_data_results.gpkg", driver="GPKG")
    results.to_parquet("data/auxiliary_data_results.parquet")
    print("Data saved successfully")

Processing points:  18%|█▊        | 511/2807 [07:47<26:52,  1.42it/s]  

No items found for point 513 (88.888888, 88.888888)


Processing points:  33%|███▎      | 934/2807 [16:56<37:29,  1.20s/it]  

No items found for point 935 (88.888888, 88.888888)


Processing points: 100%|██████████| 2807/2807 [55:20<00:00,  1.18s/it]  


Data saved successfully
